# Several Dimmers in a confined space

As a first approximation for several dimmers, I'll use open boundary conditions and place walls in the x and y boundaries. 

In [493]:
import lammps2d as lmp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anm
import matplotlib.collections as clt
import pandas as pd
import trackpy as tp

from IPython.display import HTML

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [494]:
x_loc = np.arange(-90,90+1,5)
y_loc = np.arange(-90,90+1,5)

[X,Y] = np.meshgrid(x_loc,y_loc)
Z = np.zeros(np.shape(X))

initial_positions = np.array([[x,y,z] for (x,y,z) in zip(X.flatten(),Y.flatten(),Z.flatten())])

Most of the experiments are done at this angle:

In [495]:
print("$\\theta$ = ", np.arctan2(3.3,6.5)*180/np.pi)
print("$H_0$ = ",np.sqrt(3.3**2+6.5**2))

$\theta$ =  26.916554617
$H_0$ =  7.28971878744


In [496]:
temp = 300 # K
Kb = 4/300 # pN nm / K
diffusion = 0.1e6 #nm^2/s
drag = Kb*temp/diffusion #pN/(nm/s)
sigma = 2.8

sim = lmp.sim(initial_positions,
              magnitude = 7.28,
              file_name = "test3",
              radius = sigma/2,
              diffusion = 0.14,
              frequency=1,
              susceptibility = 0.4,
              angle=27,
              framerate=1e1,
              timestep=1e-4,
              total_time = 20,
              temperature = 300,
              space = {'region':[183,183,4],'boundary':['f','f','f'],'walls':[True,True,True]})

In [497]:
%%time
sim.generate_scripts()
sim.run()
## This was about half an hour of wall time

Wall time: 9min 18s


In [498]:
%%time
trj = sim.load(read_trj=True)

Wall time: 8.28 s


In [499]:
%%time
anim = animate_trj(trj,sim)

anim.save("video3.avi",writer="ffmpeg")

Wall time: 4min 51s


In [ ]:
HTML("""
<video controls>
  <source src="video2.mp4" type="video/mp4">
</video>
""")

In [466]:
%%time
HTML(animate_trj(trj,sim).to_html5_video())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


# Support Functions

In [238]:
def plot_trj(trj):
    idx = pd.IndexSlice

    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(15,5))
    for i in trj.index.get_level_values('id').unique():
        ax1.plot(trj.loc[idx[:,i],'x'],trj.loc[idx[:,i],'y'])
        ax2.plot(trj.loc[idx[:,i],'x'],trj.loc[idx[:,i],'z'])

    ax1.set(aspect='equal');
    ax2.set(aspect='equal');
    ax2.set_ylim([-5,5])

In [436]:
def animate_trj(trj,sim,ax=False):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    
    idx = pd.IndexSlice
    
    if not ax:
        fig, ax = plt.subplots(1,1,figsize=(7,7))

    particles = trj.index.get_level_values('id').unique()
    n_of_particles = len(trj.index.get_level_values('id').unique())
    
    region = sim.sim_parameters.space["region"] 
    radius = sim.particle_properties[0].radius
    
    framerate = sim.run_parameters.framerate
    runtime = sim.run_parameters.total_time
    timestep = sim.run_parameters.timestep
    
    lammps_time = 1e6;
    
    dt_data = timestep*framerate*lammps_time # Data timestep in lammps_time
    dt_video = 1/framerate*1000 # video timestep in miliseconds
    frames = runtime*framerate+1
    
    ax.set_xlim(region[0],region[1])
    ax.set_ylim(region[2],region[3])
    ax.set(aspect='equal')
    ax.set_xlabel("$x [\mu{m}]$")
    ax.set_ylabel("$y [\mu{m}]$")
    
    patches = []
    for i,p in enumerate(particles):
        c = plt.Circle((0, 0), radius)
        patches.append(c)

    p = clt.PatchCollection(patches, cmap=plt.cm.RdBu)
    p.set_array(np.zeros(0))
    p.set_clim([region[4]+radius,region[5]-radius])
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0)
    plt.colorbar(p,label='$z [\mu{m}]$',cax=cax)

    def init():
        ax.add_collection(p)
        return p,

    def animate(frame):

        for (part_id,particle) in enumerate(particles):
            patches[part_id].center = (trj.loc[idx[frame*dt_data,particle],'x'],trj.loc[idx[frame*dt_data,particle],'y'])
        p.set_paths(patches)
        p.set_array(trj.loc[idx[frame*dt_data,:],'z'].values)
        ax.add_collection(p)
        return p,

    anim = anm.FuncAnimation(fig, animate, init_func=init,
                                   frames=int(frames), interval=dt_video, blit=True);
    plt.close(anim._fig)

    return anim